<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Working with Azure SQL via the Azure CLI

## Azure Actions Covered

* Creating, listing, and deleting Azure SQL Servers
* Creating, listing, and deleting databases on SQL Servers
* Creating and deleting firewall rules for SQL Servers
* Connecting and querying databases on SQL Servers

In this lecture, we're going to set up an Azure SQL server and database. We'll show two ways of doing this: provisioning the individual resources necessary, and then the "quick" way of using `az sql up`.

First, make sure you're logged in to Azure via the CLI.

```
$ az login
```

## Create an Azure SQL Database with Azure CLI

Next, we need to create a new resource group using the commands we used in the section on Resource Groups.

```
$ az group create --name sql-resource-group --location eastus
```

Our database needs a SQL Server resource, so let's create that first using `az sql server create`. Some important options:
* `--name` - Name of the SQL Server
* `--admin-user` - Name of the admin user
* `--admin-password` - Password for the admin user

```
$ az sql server create --name bens-server --resource-group sql-resource-group --location eastus --admin-user benadmin --admin-password testpassword123!
```

To make sure that your database is only accessible by you, we need to create a firewall around the SQL server using `az sql server firewall-rule create`. Some important options:
* `--name` - Name of the firewall rule
* `--start-ip-address` - Starting IP address of IP range to be allowed
* `--end-ip-address` - Ending IP address of the IP range to be allowed

You can find your IP address using your operating system's GUI to check your network settings. Additionally, you can use command line tools such as `hostname -I` if you're comfortably using `bash` or a similar command line interface.

**Note:** You'll likely need to modify your firewall rule/create a new one to access your database through the Azure Portal. Since the Azure Portal is web-hosted, it will have a different IP address from your local machine.

```
$ az sql server firewall-rule create --resource-group sql-resource-group --server bens-server -n AllowMyIp --start-ip-address 192.168.X.X --end-ip-address 192.168.X.X
```

Now that our server is created, let's create a new database. Remember, each SQL Server instance can contain multiple databases. Some important options:
* `--name` - Name of the database to be created
* `--sample-name` - Name of the sample schema to be populated. Right now, the only option is `AdventureWorksLT`. This is a useful option to set up a database with test/sample data.
* `--edition` - Edition component of the SKU (e.g. Basic, Standard, Premium, etc.)
* `--compute-model` - Compute model of the database
* `--family` - Compute generation component of the SKU
* `--capacity` - The capacity component of the SKU in integer numbers of DTUs or vcores.

```
$ az sql db create --resource-group sql-resource-group --server bens-server --name bens-db --sample-name AdventureWorksLT --edition GeneralPurpose --compute-model Serverless --family Gen5 --capacity 2
```

Let's check that our database was created successfully. We can do this via the Azure Portal or using `az sql db list` with the Resource Group name and the server name.

```
$ az sql db list --resource-group sql-resource-group --server bens-server
```

To interact with our newly created database, we can either use VS Code or use the Query Editor in the Azure Portal.

To query your database from VS Code:
* Install `mssql` package in VS Code
* Add MS SQL connection in VS Code - you'll need the host name, server name, admin name and password, etc.
* Query in VS Code to check - If you right-click on one of the sample tables, for example, you can do a quick "select top 1000".

To query your database from the Azure Portal:
* Log in to Azure Query Editor using your admin name and password
* Modify firewall - You may get an error due to the IP address not being included in your firewall rules. Follow the instructions on the page to modify your firewall.
* Run a query in Azure Query Editor to check that everything works.

Congratulations! You've set up an Azure SQL database using the CLI.

Now we can list and then delete the database we created.

```
$ az sql db list --resource-group sql-resource-group --server bens-server

$ az sql db delete --resource-group sql-resource-group --server bens-server --name bens-db
```

Next, let's list and delete the firewall rules.

```
$ az sql server firewall-rule list --resource-group sql-resource-group --server bens-server

$ az sql server firewall-rule delete --name AllowMyIp --resource-group sql-resource-group --server bens-server
```

Finally, we can list and then delete our SQL Servers.

```
$ az sql server list --resource-group sql-resource-group

$ az sql server delete --name bens-server --resource-group sql-resource-group
```

## Create an Azure SQL Database using `az sql up`

Azure provides a much easier way to set up a database than the steps we followed in the previous section. For this, we have to use a new CLI command.

First, create a new resource group (optional if just reusing the resource group from the previous section).

```
$ az group create --name sql-resource-group
```

Next, we can use the `az sql up` command. Some important options:
* `--server-name` - Name of the SQL Server to create
* `--database-name` - Name of the database to create
* `--admin-user` - Name of the admin user to create
* `--admin-password` - Password for the admin user

**Note:** As of this writing, there's not a way to populate the DB with sample data, such as with `--sample-name AdventureWorksLT`. The created database will be empty.

```
$ az sql up --resource-group sql-resource-group --location eastus --server-name bens-server --database-name bensdb --admin-user benadmin --admin-password testpassword123!
```

You can check that the database was created correctly via the Azure Portal or using `az sql db list`.

```
$ az sql db list --resource-group sql-resource-group --server bens-server
```

Finally, let's clean up our workspace so that we don't leave any unused resources running.

```
$ az group delete --name sql-resource-group --no-wait
```